# Using Fasttext word embeddings to search Danish Wikipedia
This notebook goes step-by-step through the following:
1. Load the Fasttext model for Danish language
2. Write a function to compute the vectorized representations of text in Danish
3. Compute vectorized representations for all the abstracts in Danish wikipedia
4. Write a function to compute the cosine distance between the vectorized representations
5. Test everything on some news headlines

** Global setup **

In [ ]:
try:
    with open("../global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

In [ ]:
from gensim.models.fasttext import FastText
import gensim.models.keyedvectors as word2vec
from gensim.test.utils import common_texts
from scipy.spatial.distance import cdist
from pathlib import Path
import re
import numpy as np

## Load the Fasttext model for Danish language
1. In the root folder of the project, create a directory "data" and a subdirectory "fasttext".
2. Download the pretrained danish fasttext model from here: 
   https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.da.zip
3. Place the files **wiki.da.bin** and **wiki.da.vec** in the folder you have created in (1).

In [ ]:
bin_path = Path("data", "fasttext", "wiki.da.bin")
model = FastText.load_fasttext_format(str(bin_path))
#print(model.most_similar('æble'))

Next steps:
1. Compute vectorized representations of all danish wikipedia abstracts
2. Compute vectorized representations of news titles
3. Find wikipedia article with minimal cosine distance from a given news title

## 1. Working with Danish wikipedia abstracts
Here we compute the vectorized representations of the Danish wikipedia abstracts using the pretrained Fasttext model that we just loaded.

### First step: load the Danish wikipedia

In [ ]:
from src.text.document_retrieval.wikipedia import Wikipedia
wikipedia = Wikipedia(
    language="Danish",
    cache_directory_url=False
)

### Second step: write a function to calculate the vectorized representation of any text

In [ ]:
def sumVectorRepresentation(text, verbose = False):
    pattern = re.compile('[^a-zA-Z0-9åÅøØæÆ ]+', re.UNICODE)
    text = pattern.sub('', text)
    words = text.lower().strip().split()
    text_vector = np.zeros(model.wv["a"].shape)
    if verbose:
        print("len: {}, words: {}".format(len(words), words))
    for i in range(len(words)):
        try:
            text_vector = text_vector + model.wv[words[i]]
        except KeyError as e:
            if verbose:
                print("i: {}, e: {}".format(i, e))
            continue
    return text_vector
    
#sumVectorRepresentation("Han sagde")

### Third step: compute vectorized representations for all Danish wikipedia abstracts
* All empty abstracts and those that do not have any alphanumeric symbols are removed and not considered.

In [ ]:
i = 0
i_max = 0
n_removed = 0
wikipedia.documents_clean = wikipedia.documents.copy()
wikipedia_abstract_vectors = []
pattern1 = re.compile('[^a-zA-Z0-9åÅøØæÆ]+', re.UNICODE)

for n in range(len(wikipedia.documents)):
    # if abstract length is zero, remove it
    try:
        if len(pattern1.sub('', wikipedia.documents[n].abstract)) == 0:
            del wikipedia.documents_clean[n - n_removed]
            n_removed = n_removed + 1
        else:
            wikipedia_abstract_vectors.append(sumVectorRepresentation(wikipedia.documents[n].abstract))
            i = i + 1
            if i_max > 0 and i > i_max:
                break
    except IndexError as e:
        print("n: {}, n_removed: {}, w.d: {}, w.d_c: {}".format(n, n_removed, len(wikipedia.documents), len(wikipedia.documents_clean)))
        

## 2. Calculate the cosine distance between the embeddings

In [ ]:
def cdist_func(A, B):
    dists = cdist(A, B, 'cosine')
    return np.argmin(dists, axis=0), dists #np.min(dists, axis=0)

## 3. Test the results on some news headlines

In [ ]:
# Variable below defines the number of relevant wikipedia articles to consider
n_wiki_matches = 3
# Variable below is an example news headline
example_title = "Hawaii: Flyv med helikopter hen over Kauai — en af verdens smukkeste øer"
# Hawaii: Flyv med helikopter hen over Kauai — en af verdens smukkeste øer
# Tyske myndigheder undersøger 95.000 biler af mærket Opel
# Salmonella fundet i kalkunbryst solgt i Aldi-butikker


# Calculate the vectorized representation
example_title_vector = sumVectorRepresentation(example_title)

cdist_result = cdist_func(wikipedia_abstract_vectors, [example_title_vector])
cdist_list = cdist_result[1] # List of all the cosine distances
cdist_list_sorted = np.sort(cdist_list, axis = 0) # Sorted list of cosine distances - to get top N matches

# Print the results
print("Example headline: {}\r\n".format(example_title))
## Print all the matches with their abstracts
for i in range(n_wiki_matches):
    result = np.where(cdist_list == cdist_list_sorted[i])
    print("{} Wikipedia article {}: \r\n Abstract: {}\r\n".format(i, 
                                                       wikipedia.documents_clean[result[0][0]],
                                                       wikipedia.documents_clean[result[0][0]].abstract))